<center>
<img src="../../img/ml_theme.png">
# Майнор "Интеллектуальный анализ данных" 
# Курс "Введение в анализ данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель ФКН НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="http://www.microsoft.com/en-us/openness/default.aspx#Ms-RL">Ms-RL</a>. Можно использовать в любых целях, но с обязательным упоминанием автора курса и аффилиации.

## Пример предсказания оскорблений в комментариях на Twitter

In [1]:
import pandas as pd

data = pd.read_csv('../../data/twitter_comments.csv')

data['fuck'] = data.Comment.map( lambda x : 1 if 'fuck' in x else 0 )
data['cunt'] = data.Comment.map( lambda x : 1 if 'cunt' in x.lower() else 0)

In [2]:
data.head()

,Insult,Date,Comment,fuck,cunt
0,1,20120618192155Z,You fuck your dad.,1,0
1,0,20120528192215Z,i really don't understand your point. It seem...,0,0
2,0,NaN,A majority of Canadians can and has been wrong...,0,0
3,0,NaN,listen if you dont wanna get married to a man ...,0,0
4,0,20120619094753Z,Các bạn xuống đường biểu tình 2011 có ôn hoà k...,0,0


In [3]:
import statsmodels.formula.api as sm

In [4]:
model = sm.logit(" Insult ~ fuck + cunt" , data).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.569625
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Insult   No. Observations:                 3947
Model:                          Logit   Df Residuals:                     3944
Method:                           MLE   Df Model:                            2
Date:                Tue, 15 Mar 2016   Pseudo R-squ.:                 0.01620
Time:                        11:26:56   Log-Likelihood:                -2248.3
converged:                       True   LL-Null:                       -2285.3
                                        LLR p-value:                 8.250e-17
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -1.1138      0.039    -28.775      0.000        -1.190    -1.038
fuck           0.8573      0.114      7.495      0.000         0.633     1.081
cunt           2.7642      1.059      2.611      0.009         0.689     4.839
==============================================================================
"""

In [5]:
bad_words_file = open('../../data/google_badlist.txt')

bad_words = set([word.strip() for word in bad_words_file.readlines()])


In [6]:
data['badwords'] = data.Comment.map( lambda x: 1 if 
                                    bad_words.intersection(set( [word for word in x.lower().split()])) else 0)

In [7]:
model = sm.logit(" Insult ~ badwords + fuck + cunt" , data).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.561294
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Insult   No. Observations:                 3947
Model:                          Logit   Df Residuals:                     3943
Method:                           MLE   Df Model:                            3
Date:                Tue, 15 Mar 2016   Pseudo R-squ.:                 0.03059
Time:                        11:26:56   Log-Likelihood:                -2215.4
converged:                       True   LL-Null:                       -2285.3
                                        LLR p-value:                 4.107e-30
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -1.2346      0.042    -29.132      0.000        -1.318    -1.152
badwords       0.8369      0.101      8.269      0.000         0.639     1.035
fuck           0.2466      0.137      1.803      0.071        -0.022     0.515
cunt           2.7309      1.063      2.568      0.010         0.647     4.815
==============================================================================
"""

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score

v = CountVectorizer(binary=False, ngram_range=(1,6), analyzer="char_wb")

v.fit(data.Comment)
X = v.transform(data.Comment)


model = LogisticRegression(penalty='l1', C=100)

cross_val_score(model, X, data.Insult)

array([ 0.81610942,  0.80319149,  0.81977186])